In [1]:
import os
os.chdir('..')

In [2]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client


cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0,1",
    protocol="ucx",
    enable_tcp_over_ucx=True,
    enable_infiniband=True,
    rmm_managed_memory=True,
    rmm_pool_size='24GB'
)
client = Client(cluster)
client

/usr/local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45527 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:45527/status,
Dashboard: http://127.0.0.1:45527/status,Workers: 2
Total threads: 2,Total memory: 502.58 GiB
Status: running,Using processes: True
Comm: ucx://127.0.0.1:52717,Workers: 2
Dashboard: http://127.0.0.1:45527/status,Total threads: 2
Started: Just now,Total memory: 502.58 GiB
Comm: ucx://127.0.0.1:43905,Total threads: 1
Dashboard: http://127.0.0.1:37321/status,Memory: 251.29 GiB
Nanny: ucx://127.0.0.1:55361,


In [3]:
# import cudf as pd
import dask
import dask_cudf
import dask.dataframe as dd
import cudf, cupy

dask.config.set({"dataframe.backend": "cudf"})

In [4]:
meta_tower = dd.read_csv('data/processed/meta_tower.csv')

meta_tower

,cell_id,lon,lat,deyang_center_flag,tower_area_code
npartitions=1,,,,,
,object,float64,float64,int64,int64
,...,...,...,...,...


In [5]:
meta_tower.size.compute()

202845

In [6]:
import hvplot.cudf

In [7]:
"""
tiles:
'CartoDark', 'CartoEco', 'CartoLight', 'CartoMidnight', 'EsriAntarcticImagery', 'EsriArcticImagery', 'EsriArcticOceanBase', 
'EsriArcticOceanReference', 'EsriDelormeWorldBaseMap', 'EsriImagery', 'EsriNatGeo', 'EsriOceanBase', 'EsriOceanReference', 
'EsriReference', 'EsriTerrain', 'EsriUSATopo', 'EsriWorldBoundariesAndPlaces', 'EsriWorldBoundariesAndPlacesAlternate', '
EsriWorldDarkGrayBase', 'EsriWorldDarkGrayReference', 'EsriWorldHillshade', 'EsriWorldHillshadeDark', 'EsriWorldLightGrayBase', 
'EsriWorldLightGrayReference', 'EsriWorldNavigationCharts', 'EsriWorldPhysical', 'EsriWorldShadedRelief', 'EsriWorldStreetMap', 
'EsriWorldTopo', 'EsriWorldTransportation', 'OSM', 'OpenTopoMap'
"""

(
    meta_tower
    .compute()
    .hvplot.points('lon', 'lat', geo=True, color='red', alpha=0.05, tiles='EsriWorldStreetMap')
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]

In [8]:
(
    meta_tower[(meta_tower['tower_area_code'] == 838)]
    .compute()
    .hvplot
    .points('lon', 'lat', geo=True, color='deyang_center_flag', alpha=0.2, tiles='EsriWorldStreetMap')
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (deyang_center_flag)

# Popularity

In [9]:
from dask_cudf import read_parquet
df = read_parquet('data/processed/201308/cdr_loc')


df.head()

,serv_id,client_nbr,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id,time,day_of_week,lon,lat,deyang_center_flag,tower_area_code
0,a5fvkt7s,5z06jq6z,5z06jq6z,838,33,838,46rc69e6,348C,21,4,104.263,31.0913,0,838
1,9fo16vqy,6hz03ag8,gcy0l5w8,838,483,838,6hz03ag8,3B93,21,3,104.709,31.2001,0,838
2,9mgzc8ht,fq343ibo,7kwzd0j0,838,39,838,fq343ibo,3488,14,5,104.290,30.9884,0,838
3,8nil2kw1,ce7t2smm,5r0c6xjn,838,25,838,ce7t2smm,390C,8,0,104.245,31.1316,0,838
4,8uu2mydh,ima05c55,dkhqlzr7,838,129,838,ima05c55,36DB,12,1,104.429,31.1813,0,838


In [10]:
df.shape[0].compute()

52009502

## tower popularity rank in all cdr

In [11]:
rank = cudf.from_pandas(
     df
    .groupby(['cell_id'])
    .size()
    .compute()
    .to_pandas()
    .rank(pct=True)
    .to_frame('popularity_rank')
 )

popularity_rank_plot = (
    cudf
    .merge(rank, meta_tower.compute(), on='cell_id')
    .hvplot
    .points('lon', 'lat', geo=True, color='popularity_rank', alpha=0.2, tiles='EsriWorldStreetMap')
)

popularity_rank_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (popularity_rank)

## tower popularity rank within 838(deyang)

In [12]:
rank = cudf.from_pandas(
     df[df['tower_area_code'] == 838]
    .groupby(['cell_id'])
    .size()
    .compute()
    .to_pandas()
    .rank(pct=True)
    .to_frame('popularity_rank')
 )

popularity_rank_plot = (
    cudf
    .merge(rank, meta_tower.compute(), on='cell_id')
    .hvplot
    .points('lon', 'lat', geo=True, color='popularity_rank', alpha=0.2, tiles='EsriWorldStreetMap')
)

popularity_rank_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (popularity_rank)

## output

In [13]:
import geckodriver_autoinstaller

geckodriver_autoinstaller.install(cwd=True)

'/home/r12323011/telecom/v0.34.0/geckodriver'

In [14]:
# import holoviews as hv
# from bokeh.io import export_svgs

# plot = hv.render(popularity_rank_plot)
# plot.output_backend = "svg"
# export_svgs(plot, filename="report/plot/tower/popularity_rank_201308.svg")

## cdr not in deyang is calling or called?

In [26]:
target = df[
    (df['tower_area_code'] != 838) &
    (
        (df['called_area_code'] == df['tower_area_code']) |
        (df['calling_area_code'] == df['tower_area_code'])
    )
]
target.shape[0].compute()

239029

In [27]:
target[target['calling_area_code'] != 838].shape[0].compute()

107091

In [28]:
target[target['called_area_code'] != 838].shape[0].compute()

131938

## cdr not in deyang but in chengdu is calling or called?

In [29]:
target = df[
    (df['tower_area_code'] == 28) &
    (
        (df['called_area_code'] == df['tower_area_code']) |
        (df['calling_area_code'] == df['tower_area_code'])
    )
]
target.shape[0].compute()

233386

In [34]:
target[
    (target['calling_nbr'] == target['client_nbr'])
    & (target['calling_area_code'] == 28)
].shape[0].compute()

53831

In [35]:
target[
    (target['calling_nbr'] == target['client_nbr'])
    & (target['called_area_code'] == 28)
].shape[0].compute()

66617

In [36]:
target[
    (target['called_nbr'] == target['client_nbr'])
    & (target['called_area_code'] == 28)
].shape[0].compute()

62239

In [37]:
target[
    (target['called_nbr'] == target['client_nbr'])
    & (target['calling_area_code'] == 28)
].shape[0].compute()

50699